# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [0]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [0]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [4]:
!pip install syft

     |████████████████████████████████| 256kB 2.8MB/s 
     |████████████████████████████████| 1.4MB 40.3MB/s 
     |████████████████████████████████| 256kB 43.4MB/s 
     |████████████████████████████████| 81kB 26.8MB/s 
     |████████████████████████████████| 204kB 45.7MB/s 
     |████████████████████████████████| 389kB 45.7MB/s 
     |████████████████████████████████| 460kB 43.5MB/s 
     |████████████████████████████████| 51kB 18.4MB/s 
     |████████████████████████████████| 266kB 32.1MB/s 
     |████████████████████████████████| 122kB 40.3MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067064 sha256=817b8a71de42a43e3947719b57384d8aced1808b64c722194a3bcf19200783b1
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=a6eac412c293574b409bc1310b3df7185f1aaff6bc4a011302358bb955e3408e
  Stored in di

In [5]:
import syft as sy

W0807 09:13:47.304866 140006083860352 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0807 09:13:47.329794 140006083860352 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [9]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob)

In [12]:
bob._objects

{6820468381: tensor([1, 2, 3, 4, 5])}

In [13]:
x

(Wrapper)>[PointerTensor | me:45659147324 -> bob:6820468381]

In [14]:
x.location == bob

True

In [15]:
x.id_at_location

6820468381

In [16]:
x.id

45659147324

In [17]:
x.owner

<VirtualWorker id:me #objects:0>

In [18]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [0]:
x

(Wrapper)>[PointerTensor | me:22743021741 -> bob:9559641851]

In [0]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [0]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
alice=  sy.VirtualWorker(hook, id = "alice")

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x_ptr = x.send(bob, alice)

In [0]:
x_ptr.get()


[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob, alice)

In [0]:
x.get(sum_results = True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [0]:
x

(Wrapper)>[PointerTensor | me:82005769872 -> bob:46038234160]

In [0]:
y

(Wrapper)>[PointerTensor | me:39558509819 -> bob:1999339871]

In [0]:
z = x + y

In [0]:
z

(Wrapper)>[PointerTensor | me:44951466334 -> bob:26458153894]

In [0]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:33864308488 -> bob:29652793788]

In [0]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [0]:
z.backward()

(Wrapper)>[PointerTensor | me:62522526668 -> bob:66455838554]

In [0]:
x = x.get()

In [0]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [0]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [0]:
input = th.tensor([[1.,1], [0,1], [1,0], [0,0]], requires_grad = True).send(bob)
target = th.tensor([[1.], [1],[0], [0]], requires_grad = True).send(bob)

In [0]:
weights = th.tensor([[0.], [0]], requires_grad = True).send(bob)

In [0]:
for i in range(10):
  
  pred = input.mm(weights)
  loss = ((pred - target)**2).sum()
  loss.backward()

  weights.data.sub_(weights.grad * 0.1)
  weights.grad *= 0
  print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


tensor(2.)


# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects()

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{60895674066: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x.child.garbage_collect_data

True

In [0]:
bob._objects

{84074371247: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x

(Wrapper)>[PointerTensor | me:44702925278 -> bob:33846413817]

In [0]:
bob._objects

{33846413817: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects

{33846413817: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [0]:
bob._objects

{33846413817: tensor([1, 2, 3, 4, 5])}

In [0]:
bob = bob.clear_objects()
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{54178271067: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [0]:
z = x + y

TensorsNotCollocatedException: ignored

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [0]:
z = x + y

TensorsNotCollocatedException: ignored

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [24]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(1.3005)
tensor(0.8547)
tensor(0.5806)
tensor(0.3985)
tensor(0.2757)
tensor(0.1923)
tensor(0.1352)
tensor(0.0958)
tensor(0.0684)
tensor(0.0492)
tensor(0.0356)
tensor(0.0259)
tensor(0.0190)
tensor(0.0140)
tensor(0.0104)
tensor(0.0077)
tensor(0.0058)
tensor(0.0043)
tensor(0.0033)
tensor(0.0025)


In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [29]:
train()

tensor(4.7209, requires_grad=True)
tensor(1.5427, requires_grad=True)
tensor(0.8123, requires_grad=True)
tensor(1.0051, requires_grad=True)
tensor(0.4187, requires_grad=True)
tensor(0.5941, requires_grad=True)
tensor(0.2403, requires_grad=True)
tensor(0.3496, requires_grad=True)
tensor(0.1391, requires_grad=True)
tensor(0.2062, requires_grad=True)
tensor(0.0807, requires_grad=True)
tensor(0.1220, requires_grad=True)
tensor(0.0469, requires_grad=True)
tensor(0.0724, requires_grad=True)
tensor(0.0273, requires_grad=True)
tensor(0.0432, requires_grad=True)
tensor(0.0159, requires_grad=True)
tensor(0.0259, requires_grad=True)
tensor(0.0094, requires_grad=True)
tensor(0.0156, requires_grad=True)
tensor(0.0055, requires_grad=True)
tensor(0.0095, requires_grad=True)
tensor(0.0033, requires_grad=True)
tensor(0.0058, requires_grad=True)
tensor(0.0020, requires_grad=True)
tensor(0.0036, requires_grad=True)
tensor(0.0012, requires_grad=True)
tensor(0.0022, requires_grad=True)
tensor(0.0007, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [30]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

<VirtualWorker id:alice #tensors:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [33]:
bob._objects

{41915717363: tensor([1, 2, 3, 4, 5])}

In [34]:
alice._objects

{35332218184: (Wrapper)>[PointerTensor | alice:35332218184 -> bob:41915717363]}

In [0]:
y = x + x

In [36]:
y

(Wrapper)>[PointerTensor | me:341121384 -> alice:43386724465]

In [37]:
bob._objects

{41915717363: tensor([1, 2, 3, 4, 5]),
 50625628528: tensor([ 2,  4,  6,  8, 10])}

In [38]:
alice._objects

{35332218184: (Wrapper)>[PointerTensor | alice:35332218184 -> bob:41915717363],
 43386724465: (Wrapper)>[PointerTensor | alice:43386724465 -> bob:50625628528]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [41]:
bob._objects

{56244535173: tensor([1, 2, 3, 4, 5])}

In [42]:
alice._objects

{88516567506: (Wrapper)>[PointerTensor | alice:88516567506 -> bob:56244535173]}

In [43]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:88516567506 -> bob:56244535173]

In [44]:
bob._objects

{56244535173: tensor([1, 2, 3, 4, 5])}

In [45]:
alice._objects

{}

In [46]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [47]:
bob._objects

{}

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [49]:
bob._objects

{16024999573: tensor([1, 2, 3, 4, 5])}

In [50]:
alice._objects

{56844711312: (Wrapper)>[PointerTensor | alice:56844711312 -> bob:16024999573]}

In [0]:
del x

In [52]:
bob._objects

{}

In [53]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [54]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [56]:
bob._objects

{48658188965: tensor([1, 2, 3, 4, 5])}

In [57]:
alice._objects

{}

In [58]:
x.move(alice)

(Wrapper)>[PointerTensor | me:2374031082 -> alice:54555754878]

In [59]:
bob._objects

{}

In [60]:
alice._objects

{54555754878: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [62]:
bob._objects

{97392879680: tensor([1, 2, 3, 4, 5])}

In [63]:
alice._objects

{32635304830: (Wrapper)>[PointerTensor | alice:32635304830 -> bob:97392879680],
 54555754878: tensor([1, 2, 3, 4, 5])}

In [64]:
x.remote_get()

(Wrapper)>[PointerTensor | me:14202370677 -> alice:32635304830]

In [65]:
bob._objects

{}

In [66]:
alice._objects

{32635304830: tensor([1, 2, 3, 4, 5]), 54555754878: tensor([1, 2, 3, 4, 5])}

In [0]:
x.move(bob)

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [0]:
x

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [0]:
bob._objects

{87205391815: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}